# Phase Query and Positional Index

In [1]:
import os
from nltk.corpus import RegexpTokenizer,stopwords

In [2]:
# current directory
curr_dir = os.getcwd()

# dataset path

path = curr_dir+'\Dataset'
print(path)
# all files
folder_name= 'C:\\Users\\anshu\\IR\\CSE508_Winter2024_A1-main\\Dataset'

#file_list = os.listdir(path)
file_list = sorted(os.listdir(path))

C:\Users\anshu\IR\CSE508_Winter2024_A1-main\Dataset


In [3]:
print(f'Total number of files : {len(file_list)}')

Total number of files : 999


In [4]:
# This would be the document structure we would be using for postings list
# This is the sample data structure
data_structure = {'word':['document_frequency', {'document_number':['no_of_occurences', ['positions_in_the_document']]}]}

In [5]:
def find_occurences(word_list,word):
    occurences = [index+1 for index in range(len(word_list)) if word_list[index]==word]
    return len(occurences),occurences

In [6]:
positional_index = {}

for file_index in range(len(file_list)):

    f = open(f'{folder_name}/{file_list[file_index]}', 'r')
    all_words = f.read().split()

    file_number = file_index + 1
  
    unique_words=list(set(all_words))

    for unique_word in unique_words:
        no_of_occurences,indices = find_occurences(all_words,unique_word)
    
        if unique_word in positional_index.keys():
            positional_index[unique_word][0] = positional_index[unique_word][0] + 1
            document_level_data_structure = positional_index[unique_word][1]
            document_level_data_structure[file_number] = [no_of_occurences,indices]
        else:
            positional_index[unique_word] = [1,{file_number:[no_of_occurences,indices]}]

In [7]:
import pickle

pickle.dump(positional_index,open('positional_index.pkl','wb'))
positional_index = pickle.load(open('positional_index.pkl','rb'))

In [8]:
for i in positional_index:
    print(i, positional_index[i], "\n\n")

strat [36, {1: [1, [5]], 56: [1, [5]], 72: [2, [50, 64]], 109: [1, [18]], 159: [1, [5]], 163: [1, [2]], 168: [2, [3, 6]], 172: [1, [24]], 274: [1, [8]], 283: [1, [57]], 313: [4, [7, 16, 19, 50]], 330: [3, [6, 46, 60]], 336: [1, [57]], 360: [1, [13]], 380: [2, [3, 35]], 396: [1, [29]], 398: [1, [4]], 411: [1, [8]], 467: [2, [8, 51]], 478: [1, [32]], 511: [2, [20, 23]], 518: [1, [5]], 533: [1, [12]], 570: [1, [72]], 586: [2, [14, 18]], 613: [2, [8, 17]], 615: [1, [27]], 658: [2, [3, 50]], 696: [1, [23]], 781: [1, [6]], 821: [1, [6]], 838: [2, [4, 8]], 890: [1, [4]], 935: [1, [4]], 976: [1, [12]], 993: [1, [35]]}] 


springs [5, {1: [2, [3, 13]], 193: [2, [3, 14]], 411: [1, [28]], 786: [1, [5]], 931: [2, [24, 51]]}] 


floating [3, {1: [1, [10]], 163: [1, [73]], 369: [1, [5]]}] 


good [204, {1: [1, [6]], 6: [1, [49]], 9: [1, [33]], 14: [1, [7]], 15: [1, [24]], 19: [1, [4]], 22: [1, [69]], 35: [2, [32, 38]], 43: [1, [2]], 48: [1, [16]], 50: [1, [25]], 62: [1, [20]], 63: [1, [13]], 65: [1,

In [9]:
def preprocess_input_query(input_query):
    stop_words = set(stopwords.words('english'))
    input_query = input_query.lower()
    updated_word_list = input_query.split()
    token = RegexpTokenizer(r"\w+")
    updated_word_list = token.tokenize(' '.join(updated_word_list))
    input_query = ' '.join(updated_word_list)
    input_query = input_query.split(" ")
    input_tokens = [word for word in input_query if word not in stop_words]

    return input_tokens

In [10]:
def intersection_of_multiple_list(l):
    final_list = []

    for i in l:
        if final_list == []:
            final_list=set(i)
        else:
            final_list = set(final_list).intersection(set(i))
    
    return sorted(list(final_list))

In [11]:
def doc_position_list_of_tokens(l1,l2):
    n1=len(l1)
    n2=len(l2)
    i=0
    j=0

    resultant_list = []

    while i<n1 and j<n2:
        if l1[i] + 1 == l2[j]:
            resultant_list.append(l2[j])
            i+=1
            j+=1
        elif l1[i] > l2[j]:
            j+=1
        elif l1[i] < l2[j]:
            i+=1
    
    return resultant_list

In [12]:
def print_postional_index_query_results(document_dict, i):
    
    document_list = [file_list[doc-1] for doc in document_dict.keys()]
    
    print(f'\nNumber of documents retrieved for query {i} query using positional index is {len(document_dict)}')
    if len(document_dict.keys()) > 0:
        print(f'Names of documents retrieved for query {i} query using positional index is {document_list}')
    else:
        print(f'Number of documents retrieved for query {i} query using positional index is {len(document_dict)}')
    print("\n")

In [13]:
n = int(input("Enter number of phrase queries to execute: "))
print(n, "\n")
query_no = 1

while n > 0:
    input_query = input("Enter the input phrase query: ")

    input_tokens = preprocess_input_query(input_query)

    doc_indices_of_tokens = []
    positional_indices_of_tokens = []

    for token in input_tokens:
        if token in positional_index.keys():
            doc_indices_of_tokens.append(list(positional_index[token][1].keys()))
            positional_indices_of_tokens.append(positional_index[token][1])

    common_documents = intersection_of_multiple_list(doc_indices_of_tokens)

    relevant_doc_ids = {}

    for doc in common_documents:
        resultant_list = []
        for i in range(len(positional_indices_of_tokens)):
            if resultant_list == []:
                resultant_list = positional_indices_of_tokens[i][doc][1]
            else:
                resultant_list = doc_position_list_of_tokens(resultant_list, positional_indices_of_tokens[i][doc][1])

            if len(resultant_list) == 0:
                continue

        if len(resultant_list) > 0:
            relevant_doc_ids[doc] = len(resultant_list)

    relevant_doc_ids = dict(sorted(relevant_doc_ids.items(), reverse=True, key=lambda x: x[1]))

    # Display output using positional index
    print(f"\nResults for Query {query_no} using positional index:")
    print(f'Number of documents retrieved: {len(relevant_doc_ids)}')
    print(f'Names of documents ID retrieved: {list(relevant_doc_ids.keys())}')

    query_no += 1
    n -= 1


Enter number of phrase queries to execute: 1
1 

Enter the input phrase query: supply power

Results for Query 1 using positional index:
Number of documents retrieved: 1
Names of documents ID retrieved: [298]


In [14]:
# extract files from the directory
import os
import re
# current directory
curr_dir = os.getcwd()

# dataset path

path = curr_dir+'\Dataset'
print(path)   

# all files
files = os.listdir(path)
print(files)

C:\Users\anshu\IR\CSE508_Winter2024_A1-main\Dataset
['file1.txt', 'file10.txt', 'file100.txt', 'file101.txt', 'file102.txt', 'file103.txt', 'file104.txt', 'file105.txt', 'file106.txt', 'file107.txt', 'file108.txt', 'file109.txt', 'file11.txt', 'file110.txt', 'file111.txt', 'file112.txt', 'file113.txt', 'file114.txt', 'file115.txt', 'file116.txt', 'file117.txt', 'file118.txt', 'file119.txt', 'file12.txt', 'file120.txt', 'file121.txt', 'file122.txt', 'file123.txt', 'file124.txt', 'file125.txt', 'file126.txt', 'file127.txt', 'file128.txt', 'file129.txt', 'file13.txt', 'file130.txt', 'file131.txt', 'file132.txt', 'file133.txt', 'file134.txt', 'file135.txt', 'file136.txt', 'file137.txt', 'file138.txt', 'file139.txt', 'file14.txt', 'file140.txt', 'file141.txt', 'file142.txt', 'file143.txt', 'file144.txt', 'file145.txt', 'file146.txt', 'file147.txt', 'file148.txt', 'file149.txt', 'file15.txt', 'file150.txt', 'file151.txt', 'file152.txt', 'file153.txt', 'file154.txt', 'file155.txt', 'file156.t

In [15]:
# Reading the "doc_id.txt" to fetch the files and document number
import json
docId = {}
with open('doc_id.txt', 'r') as file:
     docId = json.load(file)                # loading the file
docId

{'1': 'file1.txt',
 '2': 'file10.txt',
 '3': 'file100.txt',
 '4': 'file101.txt',
 '5': 'file102.txt',
 '6': 'file103.txt',
 '7': 'file104.txt',
 '8': 'file105.txt',
 '9': 'file106.txt',
 '10': 'file107.txt',
 '11': 'file108.txt',
 '12': 'file109.txt',
 '13': 'file11.txt',
 '14': 'file110.txt',
 '15': 'file111.txt',
 '16': 'file112.txt',
 '17': 'file113.txt',
 '18': 'file114.txt',
 '19': 'file115.txt',
 '20': 'file116.txt',
 '21': 'file117.txt',
 '22': 'file118.txt',
 '23': 'file119.txt',
 '24': 'file12.txt',
 '25': 'file120.txt',
 '26': 'file121.txt',
 '27': 'file122.txt',
 '28': 'file123.txt',
 '29': 'file124.txt',
 '30': 'file125.txt',
 '31': 'file126.txt',
 '32': 'file127.txt',
 '33': 'file128.txt',
 '34': 'file129.txt',
 '35': 'file13.txt',
 '36': 'file130.txt',
 '37': 'file131.txt',
 '38': 'file132.txt',
 '39': 'file133.txt',
 '40': 'file134.txt',
 '41': 'file135.txt',
 '42': 'file136.txt',
 '43': 'file137.txt',
 '44': 'file138.txt',
 '45': 'file139.txt',
 '46': 'file14.txt',
 '47